# Explainable AI (XAI)

- What
- Why
    - Trust
    - Compliance
    - Discovery
    - Understanding
- Causality vs Correlation
- Fairness
- How - theory
    - eli5
    - shap
    - lime
- How - experimentation
